In [7]:
# fullstack gpt code challenge 04
from dotenv import dotenv_values
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

config = dotenv_values(".env")

llm = ChatOpenAI(
    openai_api_key=config['OPENAI_API_KEY'],
    model="gpt-4o-mini-2024-07-18",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

# 문서 로드와 쪼개기 / 임베딩 생성 및 캐시
cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("./files/document-c04.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()

# 메모리와 문서를 이용한 프롬프트
memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=20,
    return_messages=True,
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

# 체인 연결 / 체인 호출 함수 정의
chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory,
    }
    | prompt
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    print(result)
    memory.save_context({"input": question}, {"output": result.content})

In [8]:
invoke_chain("Is Aaronson guilty?")

According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, despite having a memory that contradicts this belief. He recalls having seen a photograph that disproved their guilt, but he convinces himself that it never existed and that he invented it. Therefore, in the context provided, Winston accepts that Aaronson is guilty.content='According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, despite having a memory that contradicts this belief. He recalls having seen a photograph that disproved their guilt, but he convinces himself that it never existed and that he invented it. Therefore, in the context provided, Winston accepts that Aaronson is guilty.'


In [9]:
invoke_chain("What message did he write in the table?")

Winston wrote "FREEDOM IS SLAVERY" and beneath it "TWO AND TWO MAKE FIVE" on the table. He also wrote "GOD IS POWER."content='Winston wrote "FREEDOM IS SLAVERY" and beneath it "TWO AND TWO MAKE FIVE" on the table. He also wrote "GOD IS POWER."'


In [10]:
invoke_chain("Who is Julia?")

Julia is a character who has a romantic relationship with Winston. She is portrayed as a rebellious figure against the Party, engaging in a love affair with Winston despite the oppressive regime they live under. Julia represents a sense of individuality and personal freedom in contrast to the totalitarian control of the Party.content='Julia is a character who has a romantic relationship with Winston. She is portrayed as a rebellious figure against the Party, engaging in a love affair with Winston despite the oppressive regime they live under. Julia represents a sense of individuality and personal freedom in contrast to the totalitarian control of the Party.'


In [11]:
load_memory({})

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content='According to the context, Winston believes that Aaronson, along with Jones and Rutherford, is guilty of the crimes they are charged with, despite having a memory that contradicts this belief. He recalls having seen a photograph that disproved their guilt, but he convinces himself that it never existed and that he invented it. Therefore, in the context provided, Winston accepts that Aaronson is guilty.'),
 HumanMessage(content='What message did he write in the table?'),
 AIMessage(content='Winston wrote "FREEDOM IS SLAVERY" and beneath it "TWO AND TWO MAKE FIVE" on the table. He also wrote "GOD IS POWER."'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content='Julia is a character who has a romantic relationship with Winston. She is portrayed as a rebellious figure against the Party, engaging in a love affair with Winston despite the oppressive regime they live under. Julia represents a sense of individuality and p